In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data = None

column_names = ['fruit', 'long', 'sweet', 'yellow', 'seed', 'Brazil']

with open('../fruit_data.pkl', 'rb') as fin:
    data = pickle.load(fin)

In [3]:
data

array([['others', '0', '1', '0', '1', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ...,
       ['bananas', '1', '1', '1', '0', '0'],
       ['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0']], dtype='<U21')

In [4]:
df = pd.DataFrame(data, columns=column_names)
df

,fruit,long,sweet,yellow,seed,Brazil
0,others,0,1,0,1,0
1,bananas,1,0,1,0,0
2,bananas,1,0,1,0,0
3,bananas,0,1,1,0,1
4,oranges,0,1,0,1,0
...,...,...,...,...,...,...
1395,oranges,0,1,0,1,0
1396,mangoes,0,0,0,0,1
1397,bananas,1,1,1,0,0
1398,oranges,0,1,0,1,0


In [5]:
df.long = pd.to_numeric(df.long)
df.sweet = pd.to_numeric(df.sweet)
df.yellow = pd.to_numeric(df.yellow)
df.seed = pd.to_numeric(df.seed)
df.Brazil = pd.to_numeric(df.Brazil)

In [22]:
def train(df):
    
    priors = calculate_prior(df)
    posteriors = calculate_posterior(df, priors)
    
    return priors, posteriors

In [23]:
def calculate_prior(df):
    return df.fruit.value_counts()/df.shape[0]

In [24]:
def calculate_posterior(df, priors):
    column_names = ['long', 'NOT_long', 'sweet', 'NOT_sweet', 'yellow', 'NOT_yellow', 'seed', 'NOT_seed', 'Brazil', 'NOT_Brazil']
    posteriors = pd.DataFrame(np.zeros([4, 10]), index=priors.index, columns=column_names)
    
    # iterating over every fruit and calculating its posterior for the LONG and NOT LONG features
    for feature in df.columns:

        if feature != 'fruit':

            feature_count = []
            not_feature_count = []

            for fruit in posteriors.index:

                total_fruit_count = df[df.fruit == fruit].shape[0]
                feature_count.append(df[df.fruit == fruit][feature].sum() / total_fruit_count)
                not_feature_count.append((df[df.fruit == fruit].shape[0] - df[df.fruit == fruit][feature].sum()) / total_fruit_count)

            posteriors[feature] = feature_count
            posteriors['NOT_' + feature] = not_feature_count
    return posteriors

In [26]:
priors, posteriors = train(df)